## This notebook shows the canonical mean to select some spots with a simple filter
* It is possible to interact with external libraries, have a look on the `init_diagram` notebook.
* To instanciate a `Diagram` allows you to apply efficiently filters based on image processing.
* They are several means to instanciate a `diagram` some of them are betters than overs.

In [ ]:
%matplotlib notebook

import multiprocessing.pool
import pathlib

import matplotlib.pyplot as plt
import torch
import tqdm

from laueimproc.io.download import get_samples  # gives access to the dataset
from laueimproc import Diagram

In [ ]:
"""Define some utils"""

def select_items(criteria: torch.Tensor, threshold: float, nb_min: int) -> torch.Tensor:
    """Select a least `nb_min` items and also `criteria` >= `threshold`."""
    if len(criteria) <= nb_min:
        return torch.arange(len(criteria))
    indexs = torch.argsort(confidence, descending=True)  # the indexs from the best to the worse
    threshold = min(threshold, indexs[nb_min-1].item())
    return criteria >= threshold

## Set and take control of the global configuration parameters
* The cache is automaticaly managed, but you can control it a bit.

In [ ]:
# manage context

from laueimproc.opti.manager import DiagramManager

manager = DiagramManager()
manager.verbose = True  # to display the operations in background
manager.max_mem_percent = 80  # trigger cache clear after 80% of ram full, default based on swapiness
manager.update()  # dereference unreachable diagrams

## Initialise the diagrams
* The different ways of initializing a diagram are described in detail in the `init_diagram` notebook.

In [ ]:
all_files = sorted(get_samples().iterdir())  # the list of all images path
# all_files = list(pathlib.Path("/data/visitor/a322855/bm32/20240221/RAW_DATA/Almardi/Almardi_map2DGemardi3_GOOD_0004/scan0002").iterdir())
diagrams = [Diagram(f) for f in all_files]
for diagram in diagrams:
    diagram.find_spots()
    # diagram.plot(plt.figure(layout="tight", figsize=(8, 8))); plt.show()

### Basic filtering

In [ ]:
"""Sorted by intensity"""

for diagram in tqdm.tqdm(diagrams, smoothing=0.01):
    intensities = diagram.compute_pxl_intensities()
    sorted_indexs = torch.argsort(intensities, descending=True)
    diagram.filter_spots(sorted_indexs, msg="sorted by intensities", inplace=True)
    # print(diagram)
    # diagram.plot(plt.figure(layout="tight", figsize=(8, 8))); plt.show()

### Fit spots roi with gaussians

In [ ]:
for diagram in tqdm.tqdm(diagrams, smoothing=0.01):
    mean, cov, infodict = diagram.fit_gaussian(photon_density=10900.0, tol=True)
    confidence = 3.0 * infodict["tol"]  # 99% confidence interval on pixel position
    selection = select_items(-confidence, -0.5, 10)  # 10 best and position +- 0.5pxl
    diagram.filter_spots(selection, msg="remove spots with uncertain position", inplace=True)
    # print(diagram)
    # diagram.plot(plt.figure(layout="tight", figsize=(8, 8))); plt.show()